# Goal

Save N4 bias field correction

# Imports

In [7]:
import os

# Paths to (1) code (2) data (3) saved models
code_src    = "/gpfs/home/gologr01"
data_src    = "/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata"
model_src   = "/gpfs/data/oermannlab/private_data/DeepPit/saved_models"

# UMich 
# code src: "/home/labcomputer/Desktop/Rachel"
# data src: "../../../../..//media/labcomputer/e33f6fe0-5ede-4be4-b1f2-5168b7903c7a/home/rachel/"

deepPit_src = f"{code_src}/DeepPit"
obelisk_src = f"{code_src}/OBELISK"
label_src   = f"{data_src}/samir_labels"
ABIDE_src   = f"{data_src}/ABIDE"

# print
print("Folders in data src: ", end=""); print(*os.listdir(data_src), sep=", ")
print("Folders in label src (data w labels): ", end=""); print(*os.listdir(label_src), sep=", ")
print("Folders in ABIDE src (data wo labels) ", end=""); print(*os.listdir(ABIDE_src), sep=", ")

Folders in data src: ICMB, ABVIB (1).zip, central.xnat.org, ADNI, PPMI, Oasis_long, samir_labels, ACRIN-FMISO-Brain, LGG-1p19qDeletion, REMBRANDT, AIBL, test.txt, CPTAC-GBM, ABIDE-ABIDE-top.txt, TCGA-GBM, TCGA-LGG, ABVIB, ABIDE, AIBL.zip
Folders in label src (data w labels): 50155-50212, 50313-50372, 50213-50312, 50373-50453, 50002-50153
Folders in ABIDE src (data wo labels) PAD, ABIDE_1, ABIDE


In [12]:
# imports

import os
import sys
import time
import glob
# sys.path.append('/gpfs/home/gologr01/DeepPit')
# sys.path.append('/gpfs/home/gologr01/OBELISK')

# imports
import SimpleITK as sitk
import torch
import meshio
from pathlib import Path
from helpers.preprocess import seg2mask, get_data_dict

# MR data

In [29]:
def get_data_dict_n4(train_path):
    train_folders   = os.listdir(train_path)
    train_data_dict = {}
    for folder in train_folders:
        segm_obj_path = os.path.join(train_path, folder, "seg.pt")

        mp_path      = os.path.join(train_path, folder, "MP-RAGE")
        folder1_path = os.path.join(mp_path, os.listdir(mp_path)[0])
        folder2_path = os.path.join(folder1_path, os.listdir(folder1_path)[0])

        # choose corrected_n4 if available
        nii_paths = glob.glob(f"{folder2_path}/*.nii")
        nii_path = nii_paths[0]
         
        if len(nii_paths) > 1 and not nii_path.endswith("corrected_n4.nii"):
            nii_path = nii_paths[1]
            
        if len(nii_paths) > 2:
            print(folder2_path)
            
        train_data_dict[folder] = (nii_path, segm_obj_path) #(segm_obj_path, nii_path)
    return train_data_dict

In [30]:
# Get data dict
data = {}
folders = os.listdir(label_src)
for folder in folders: 
    data.update(get_data_dict_n4(f"{label_src}/{folder}"))

# Convert data dict => items (path to MR, path to Segm tensor)
items = list(data.values())

# filter no corrected
items_no_n4 = [item for item in items if not item[0].endswith("corrected_n4.nii")]

/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50201/MP-RAGE/2000-01-01_00_00_00.0/S164577
/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50187/MP-RAGE/2000-01-01_00_00_00.0/S165254
/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50158/MP-RAGE/2000-01-01_00_00_00.0/S165723
/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50168/MP-RAGE/2000-01-01_00_00_00.0/S165679
/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50313-50372/50365/MP-RAGE/2000-01-01_00_00_00.0/S165210


In [31]:
print(len(items), len(items_no_n4), len(items)-len(items_no_n4))

335 267 68


In [15]:
print(items[0][0])
print(items[0][0][:-4] + "_corrected_n4.nii")

/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50201/MP-RAGE/2000-01-01_00_00_00.0/S164577/ABIDE_50201_MRI_MP-RAGE_br_raw_20120830171150028_S164577_I328580.nii
/gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50201/MP-RAGE/2000-01-01_00_00_00.0/S164577/ABIDE_50201_MRI_MP-RAGE_br_raw_20120830171150028_S164577_I328580_corrected_n4.nii


# Process

In [36]:
# from FAIMED3D 02_preprocessing
# and https://simpleitk.readthedocs.io/en/master/link_N4BiasFieldCorrection_docs.html

# 45-50 secs each => 5 hrs
start = time.time()

print(f"N left: ", len(items_no_n4))

count = 0

for mr_path, seg_path in items_no_n4: 
    
    start1 = time.time()
    
    # Read in image
    inputImage = sitk.ReadImage(mr_path, sitk.sitkFloat32)
    
    # Set corrector
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    corrector.SetMaximumNumberOfIterations([3] * 3)
    corrected_image = corrector.Execute(inputImage) # mask_image

    # write image
    corrected_fn = mr_path[:-4] + "_corrected_n4.nii"
    sitk.WriteImage(corrected_image, corrected_fn)

    elapsed1 = time.time() - start1
    print(f"Elapsed {elapsed1} s")

    print(f"Index {count}, fn {corrected_fn}")
    count += 1

elapsed = time.time() - start
print(f"Total Elapsed {elapsed} s")

Elapsed 41.68631362915039 s
Index 0, fn /gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50201/MP-RAGE/2000-01-01_00_00_00.0/S164577/ABIDE_50201_MRI_MP-RAGE_br_raw_20120830171150028_S164577_I328580_corrected_n4.nii
Elapsed 41.72786521911621 s
Index 1, fn /gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50193/MP-RAGE/2000-01-01_00_00_00.0/S164730/ABIDE_50193_MRI_MP-RAGE_br_raw_20120830181246062_S164730_I328740_corrected_n4.nii
Elapsed 55.79793906211853 s
Index 2, fn /gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50160/MP-RAGE/2000-01-01_00_00_00.0/S165701/ABIDE_50160_MRI_MP-RAGE_br_raw_20120831135512895_S165701_I329747_corrected_n4.nii
Elapsed 46.6309449672699 s
Index 3, fn /gpfs/data/oermannlab/private_data/DeepPit/PitMRdata/samir_labels/50155-50212/50164/MP-RAGE/2000-01-01_00_00_00.0/S165632/ABIDE_50164_MRI_MP-RAGE_br_raw_20120831123146452_S165632_I329676_corrected_n4.nii
Elapsed 41.7067186832428 

KeyboardInterrupt: 